In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [ ]:
churn_df = pd.read_csv("../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")
churn_df.head()

In [ ]:
#check missing values
churn_df.isna().sum()

In [ ]:
churn_df.dtypes

In [ ]:
#in the above cell it is clear that TotalCharges is object type even if it looks like floats in the table. 
#It's better to fix this.
churn_df['TotalCharges'] = churn_df['TotalCharges'].replace({" ":'0'})
churn_df['TotalCharges'] = churn_df['TotalCharges'].astype(float)

In [ ]:
#let's explore our features
import pandas_profiling

profile = pandas_profiling.ProfileReport(churn_df)
profile

In [ ]:
sns.countplot(x ='gender', hue='Churn', data=churn_df)

In [ ]:
sns.countplot(x ='SeniorCitizen', hue='Churn', data=churn_df)

In [ ]:
sns.countplot(x ='Dependents', hue='Churn', data=churn_df)

In [ ]:
sns.countplot(x ='Partner', hue='Churn', data=churn_df)

So far we have seen that there are no missing values.
The graph showed that there are no correlation between churn and gender - the numbers are roughly the same.
There are less senior citizens than younger ones and they (seniors) tend to churn almost as much as not churn, while youngest churn less.
If they are dependants, they are less likely to be churned.
Partners are less likely to be churned as well.

Now, let's check predictive Power Score and see which features influence Churn.

In [ ]:
pip install ppscore

In [ ]:
def heatmap(df):
    df = df[['x', 'y', 'ppscore']].pivot(columns='x', index='y', values='ppscore')
    fig, ax = plt.subplots(figsize=(20,20)) 
    ax = sns.heatmap(df, vmin=0, vmax=1, cmap="Blues", linewidths=0.5, annot=True)
    ax.set_title("PPS matrix")
    ax.set_xlabel("feature")
    ax.set_ylabel("target")
    return ax

In [ ]:
import ppscore as pps
matrix = pps.matrix(churn_df)
heatmap(matrix)

Here we can clearly see that there is relationship between such features as MonthlyCharges, TotalCharges, tenure and Churn. These 3 features can be predicted using almost all features from the dataset. Only Gender and CustomerID are safe to be deleted. 
Let's convert the rest of features from object to numbers and delete unnecessary features.

In [ ]:
for column in churn_df.columns:
    if churn_df[column].dtype == np.number:
        continue
    churn_df[column] = LabelEncoder().fit_transform(churn_df[column])

In [ ]:
cleaned_df = churn_df.drop(['customerID', 'gender'], axis=1)
cleaned_df

Now it's time to split the data into train and test set and build models. 
Going to use RFC, Logistic Regression and XGBoost.
As a metric - classification report - F1 score, precision, recall and accuracy.
And two additional metrics - log loss (which is the closer the loss to 0 - the better) and jaccard score (the closer to 1 - the better).

In [ ]:
x = cleaned_df.drop('Churn', axis=1)
y = cleaned_df['Churn']
x = StandardScaler().fit_transform(x)
xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
model_RFC = RandomForestClassifier(n_estimators=300, max_depth=6, random_state=42)
model_RFC.fit(xtrain, ytrain)
predictions_RFC = model_RFC.predict(xtest)
print(classification_report(ytest, predictions_RFC))

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(xtrain, ytrain)
predictions = model.predict(xtest)
print(classification_report(ytest, predictions))

In [ ]:
from sklearn.metrics import log_loss
predictions_proba = model.predict_proba(xtest)
log_loss(ytest, predictions_proba)

In [ ]:
proba_RFC = model_RFC.predict_proba(xtest)
log_loss(ytest, proba_RFC)

In [ ]:
from sklearn.metrics import jaccard_score
jaccard_score(ytest, predictions)

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(use_label_encoder=False)
xgb.fit(xtrain, ytrain)
predictions_xgb = model.predict(xtest)
print(classification_report(ytest, predictions_xgb))

In [ ]:
jaccard_score(ytest, predictions_xgb)

In [ ]:
xgb_proba = xgb.predict_proba(xtest)
log_loss(ytest, xgb_proba)

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(ytest, predictions_xgb, labels=[1,0]))
#LR showed the same results as XGBoost

In [ ]:
print(confusion_matrix(ytest, predictions_RFC, labels=[1,0]))

Jaccard loss and classification report parameters are the same for Logistic Regression and XGBoost (and a bit worse for RFC), but the log loss is the lowest for Logistic Regressin. I would go with LR.